In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re

## DIY Crawler
Extracting list of time series articles and their links from https://machinelearningmastery.com/category/time-series

* Having issues reading more than one page value.  JSON returned is repeating first page articles only.  
* Errors reading in json.  Worked before I tried doing next_page

In [2]:
class MLMSpider(scrapy.Spider):
    name = 'MLM'
    start_urls = [
        'https://machinelearningmastery.com/category/time-series/'
    ]
    
    def parse(self, response):
        for article in response.xpath('//article'):
            yield {
                'name': article.xpath('header/h2/a/@title').extract(),
                'link': article.xpath('header/h2/a/@href').extract()
            }
    
        next_page = response.xpath("//div/a/@class='next page-numbers'").extract() #issue is here
              
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
        
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'MLMfirstpage.json',
    'LOG_ENABLED': False,
})

process.crawl(MLMSpider)
process.start()
print('done!')

done!


In [3]:
import pandas as pd
pd.read_json('MLMfirstpage.json', orient='records',) #lines=True)


ValueError: Trailing data